In [1]:
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import numpy as np
import mat73

In [2]:
mat_data = mat73.loadmat('../../indy_20161005_06.mat')

In [3]:
def calculate_direction_speed_acceleration(target_pos, cursor_pos, time_points):
    # 计算位移向量
    displacement_vector = target_pos - cursor_pos
    
    # 计算时间差
    time_diff = np.diff(time_points.flatten())  # 将时间点展平，然后计算差分

    # 计算速度向量
    velocity_vector = np.diff(cursor_pos, axis=0) / time_diff[:, np.newaxis]

    # 计算加速度向量
    acceleration_vector = np.diff(velocity_vector, axis=0) / time_diff[1:, np.newaxis]

    # 计算方向角
    direction_angle = np.arctan2(displacement_vector[:, 1], displacement_vector[:, 0])
    
    angle = np.degrees(direction_angle)
    
    # 将角度调整为在0到360度之间
    direction_angle = (angle + 360) % 360

    return direction_angle, velocity_vector, acceleration_vector

In [4]:
direction_angle, velocity_vector, acceleration_vector = calculate_direction_speed_acceleration(mat_data['target_pos'], mat_data['cursor_pos'], mat_data['t'])

In [5]:
def get_R2_velocity_percent(mat_data):
    def get_total_spikes():
        spikes_times = []
        for i in range(96):
            if i == 44:
                continue
            spike_time = mat_data['spikes'][i][0]
            if spike_time is None:
                continue
            spikes_times.append(spike_time)
        return spikes_times
        
    total_spikes_times = get_total_spikes()
    
    def round_to_nearest(x, interval=0.004):
        return [int(round(num / interval) * 4) for num in x]

    def round_to_half(x):
        return [int(round(num * 2)) / 2 for num in x]

    def get_flat_data(spikes_times):
        flat_data = []
        for sublist in spikes_times:
            if sublist is not None:
                if not isinstance(sublist, list):
                    continue
                if sublist.size == 1:
                    flat_data.append(sublist.item())
                else:
                    for item in sublist:
                        flat_data.append(item)
                        
        flat_data = round_to_nearest(flat_data)
        return flat_data
    
    def get_count_histo(flat_data):
        interval = 200
        mapped_t = np.arange(1288000, 1288000 + 93501 * 4, interval)
        count_per_interval = np.histogram(flat_data, bins=1871)
        
        return count_per_interval
        
    def get_speed_half():
        velocity_vector_xy = np.array(velocity_vector)
        speeds_xy = np.linalg.norm(velocity_vector_xy, axis=1)
        speed_half = round_to_half(speeds_xy / 100)
        
        return speed_half
    
    def get_distribution_velocity(speed_half, count_per_interval):
        count_v = [set() for _ in range(13)]
        count_vs = [0 for _ in range(13)]

        repeated_values = np.repeat(np.arange(0, 1871, 1), 50)

        for i, speed in enumerate(speed_half):
            count_v[int(speed / 0.5)].add(int(repeated_values[i]))
            count_vs[int(speed / 0.5)] = count_vs[int(speed / 0.5)] + 1

        number_v = [0 for i in range(13)]

        for i in range(13):
            tmp = list(count_v[i])
            
            count = 0
            for j in tmp:
                count = count + count_per_interval[0][j]
                
            number_v[i] = count

        vv = [0 for _ in range(13)]

        for i, k in enumerate(number_v):
            vv[i] = int(number_v[i] * 2 / count_vs[i])
        
        return number_v, count_vs
    
    def pipeline(data):
        flat_data = get_flat_data(data)
        count_per_interval = get_count_histo(flat_data)
        
        speed_half = get_speed_half()
        number_v, count_vs = get_distribution_velocity(speed_half, count_per_interval)
        
        return number_v, count_vs
    
    numberv1, _ = pipeline(total_spikes_times)
    
    def get_split_data():
        spike_times = []

        for i in range(96):
            tmp = mat_data['spikes'][i][0]
            if tmp is None:
                continue
            if isinstance(tmp, float):
                continue
            if tmp.size == 1:
                continue
                # spike_times[i].append(tmp.item())
            elif tmp is not None:
                if len(tmp) > 100:
                    spike_times.append(tmp.tolist())
        
        return spike_times
    
    split_data = get_split_data()
    
    for data in split_data:
        numberv2, _ = pipeline(data)
        print(r2_score(numberv1[-4:], numberv2[-4:]))
    
    # ratio = numberv1 / np.sum(numberv1)

    
    return numberv1, numberv2

In [6]:
get_R2_velocity_percent(mat_data)
# R =0.7890
# R =-0.3791
# R =0.4230
# R =0.4328
# R =0.0478
# R =0.0339

# R =0.2108
# R =0.6804
# R =0.3829
# R =0.5922
# R =0.2033
# R =0.2578

# R =0.4776
# R =0.3924
# R =0.2723
# R =0.4406
# R =-0.0841
# R =0.0620


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# R =0.7073
# R =0.8017
# R =0.5188
# R =0.4836
# R =0.1223
# R =0.0243

In [ ]:
# R =0.5483
# R =0.4817
# R =0.1999
# R =0.3500
# R =0.0476
# R =0.0709

In [ ]:
# R =0.5049
# R =0.4702
# R =0.3952
# R =0.3070
# R =0.0854
# R =0.0147